In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import re


df = pd.read_excel("concrete.xlsx")
#df.head()
#df.columns = df.columns
col_list = df.columns.to_list()

def extract_words_before_braces(strings):
    extracted_words = []
    for string in strings:
        match = re.search(r'(.+?)\s*\(', string)
        if match:
            word = match.group(1)
            word = word.title()
            word = word.replace(" ", "")
            extracted_words.append(word)
    return extracted_words


df.columns = extract_words_before_braces(col_list)
print(df.columns)

Index(['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'Superplasticizer',
       'CoarseAggregate', 'FineAggregate', 'Age',
       'ConcreteCompressiveStrength'],
      dtype='object')


In [3]:
X = df.copy()
y = X.pop("ConcreteCompressiveStrength")

# Train and score baseline model
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 8.232


In [4]:

# Create synthetic features
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]

In [5]:
# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: 7.948
